In [20]:
# load tidy dataset

import pandas as pd
import os.path
import numpy as np
from itertools import combinations
import glob
import sys
import time

#import numpy as np
#import loadTidyCSV.py

def loadTidy(tidyData): 
    assert os.path.isfile(tidyData), "desired file does not exist" 
    df = pd.read_csv(tidyData, header = None, names = ["date", "animal", "session", "trial", "stimulus", "neuronID", "timePt", "CaSignal"])
    return df 

# helper functions to return the number of distinct types in the provided data frame
getNumNeurons = lambda df: len(np.unique(df['neuronID'].tolist()))
getNeurons = lambda df: np.unique(df['neuronID'].tolist())

getNumTrials = lambda df: len(np.unique(df['trial'].tolist()))
getTrials = lambda df: np.unique(df['trial'].tolist())


# pass in pre-filtered data set containing data for only one animal and the same session (ie SAME NEURONS!)
def getListsOfTrialIDs(df_animalSession):
    ## get trials for both stimuli
    df_anmlSessStimA = df_animalSession[df_animalSession['stimulus'] == stimA]
    df_anmlSessStimB = df_animalSession[df_animalSession['stimulus'] == stimB]
    print(stimA,stimB)

    ## get lists of trial numbers of each stimuli's presentations 
    trials_stimA = np.unique(df_anmlSessStimA['trial'].tolist())
    trials_stimB = np.unique(df_anmlSessStimB['trial'].tolist())
    print("trial IDs for each stimulus type",trials_stimA,trials_stimB)
    return (trials_stimA,trials_stimB)

## pass in a data frame with only a single animal and session    
def getNumTimePtsPerTrial(df_animalSession, trials_stimA, trials_stimB):

    #### get number of timePts in each trial selected above 
    ## (1 to 3 presentations of the same stimuli exist per session in Prabhat's data)
    numTimePtsPerTrial = np.empty((2,max(len(trials_stimA),len(trials_stimB))))
    numTimePtsPerTrial[:] = np.nan
    stimInd = 0;
    for thisStimTypeTrialNums in [trials_stimA, trials_stimB]:
        trialInd = 0
        for trial in thisStimTypeTrialNums:
            inds_thisTrial = (df_animalSession['trial']==trial)
            tmp_df_thisTrial = df_animalSession[inds_thisTrial] # gives all time points for all neurons
            numNeurons = getNumNeurons(tmp_df_thisTrial) 
            numTimePtsPerTrial[stimInd,trialInd] = np.sum(inds_thisTrial)/numNeurons 
            trialInd += 1
        stimInd += 1
    print(numTimePtsPerTrial) # rows are for stimuli type; cols are presentation of that stimulus
    return numTimePtsPerTrial

## test candidate comparisons based on whether the number of trials per session and approximate number of timePts match
def areNumTrialsPerStimulusEqual(numTimePtsPerTrial):
    
    ## no trials of either type --> discard this comparison for this animal/session   
    if np.all(np.isnan(numTimePtsPerTrial)):
        print("DISCARDED: neither stimulus type were found for this animal and session")
        return False  # skip to next session (WORK: handle this)
        
    ## different numbers of trials per stimuli/session --> discard this comparison for this animal/session 
    elif np.any(np.isnan(numTimePtsPerTrial)): 
        print("DISCARDED: mismatching numbers of trials per stimulus type for this animal/session")
        return False # skip to next session (WORK: handle this)

    ## FULFILLED here: condition that allows analysis to proceed to attempted data
    elif not np.any(np.isnan(numTimePtsPerTrial)): 
        print("trial numbers match")
    else:
        raise RuntimeError('unexpected trial comparison occurred')
        return False
    
    print("checking approx num of time points")

## input argument generated from getNumTimePtsPerTrial
def areNumTimePtsPerTrialSimilar(numTimePtsPerTrial):
    minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)             
    if (np.abs(minTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))) \
        or (np.abs(maxTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))): 
        print("DISCARDED: variance in trial length is above the user's threshold")
        return False # skip to next session (WORK: handle this)

    ### passed all criteria if it made it this far
    return True

## input argument created by getNumTimePtsPerTrial function
def timePtStats(numTimePtsPerTrial):
    minTPs = int(np.amin(numTimePtsPerTrial))
    maxTPs = int(np.amax(numTimePtsPerTrial))
    meanTPs = np.mean(numTimePtsPerTrial)
    stdTPs = np.std(numTimePtsPerTrial)
    
    ## useful for debugging
#     print('min', minTPs)
#     print('max', maxTPs)
#     print('std', stdTPs)
#     print('mean',meanTPs)
#     print('|min-mean|=',np.abs(minTPs-meanTPs))
#     print('|max-mean|=',np.abs(maxTPs-meanTPs))
#     print('|mean-std|=',np.abs(meanTPs-stdTPs))
#     print('thresh * |mean-std|=',(threshTPs_stdFromMean * np.abs(meanTPs-stdTPs)))
    
    return minTPs, maxTPs, meanTPs, stdTPs

In [2]:
################ concatenate all .csv files exported from matlab into single pandas dataframe df

# tidy csv file and dir (use makeTidy_Anderson.m to convert Ann's structure to csv)
tidyDataDir = '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/'
tidyDataFileTemplate = 'mouse'
tidyDataFileExt = '.csv'

# get all input files you want to add to the same dataset
dataFiles = np.sort(glob.glob(\
    "/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse*.csv"))
print("data files: \n", dataFiles)


print("\n loading and appending to prior pandas data frame")
dataLst = []
for file in dataFiles:
    print(file)
    dataLst.append(loadTidy(file))
df = pd.concat(dataLst)

print('finished loading')

############# ALL DATA STORED HERE IN DF
    

data files: 
 [ '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse4.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse5.csv']

 loading and appending to prior pandas data frame
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAn

In [3]:
## survey/search data to prepare for split operation
metaStrs = [['dates','date'],['animals','animal'],['sessions','session'],['maxTrials','trial'],['stimuli','stimulus']]

print("searching over entire data set to get range of various IDs for data (used in subsequent loops)") 

## this dictionary holds useful info regarding the range of inputs to loop/search over subsequently
metaDct = {}
for a,b in metaStrs:
    print(a,b)
    metaDct[a] = np.unique(df[b].tolist())
print(metaDct)


searching over entire data set to get range of various IDs for data (used in subsequent loops)
dates date
animals animal
sessions session
maxTrials trial
stimuli stimulus
{'dates': array(['2017_05_00'], 
      dtype='<U10'), 'animals': array([1, 3, 4, 5, 7]), 'sessions': array([1, 2, 3]), 'maxTrials': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21]), 'stimuli': array(['USS', 'baseline', 'female', 'male', 'mineral oil odor',
       'peanut odor', 'pred odor', 'rat', 'tone', 'toy'], 
      dtype='<U16')}


In [4]:
######### USER PARAMETERS #########
threshTPs_stdFromMean = 0.70

In [ ]:
# get all pairs of stimuli
stimCmbTpl = tuple(combinations(metaDct['stimuli'],2)) 

######### MAIN LOOP ##########
totalNumComparisons = 0;
for (stimA, stimB) in stimCmbTpl:
    print((stimA,stimB))
    totalNumComparisons += 1
    
    ## get all data for both trial types
    indsBoth = (df['stimulus']==stimA) | (df['stimulus']==stimB)
    df_bothStimuli = df[indsBoth]

    #### select data by animals and sessions
    for animal in metaDct['animals']:
        print("stimuli comparison num: ", totalNumComparisons)
        print('animal: ', animal)
        for session in metaDct['sessions']:
            print('session:', session)
            
            ## return subselection of data where the same neurons were recorded
            inds_animalSession = (df_bothStimuli['animal'] == animal) & (df_bothStimuli['session'] == session)
            df_animalSession = df_bothStimuli[inds_animalSession]
            
            # get lists of trial IDs matching stimuli
            trials_stimA, trials_stimB = getListsOfTrialIDs(df_animalSession)
            
            #### skip this comparison <-- if the data don't match in number of trials 
            numTimePtsPerTrial = getNumTimePtsPerTrial(df_animalSession,trials_stimA,trials_stimB)
            if areNumTrialsPerStimulusEqual(numTimePtsPerTrial)==False:
                break     
                
            #### skip this comparison <-- if the data don't match in approx number of timePts
            if areNumTimePtsPerTrialSimilar(numTimePtsPerTrial) == False:
                break
                
            ## truncate longer trials to shortest trial and save to new df
            minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)
            truncLst = []
            for trial in np.concatenate((trials_stimA,trials_stimB)):
                tmp_inds_trunc = (df_animalSession['trial']==trial) & (df_animalSession['timePt'] <= minTPs)
                tmp_df_trunc = df_animalSession[tmp_inds_trunc]
                truncLst.append(tmp_df_trunc)
            df_trunc = pd.concat(truncLst)
                    
            #### concatenate same cells 
            
            ### loop over and concatenate neurons into the same row if they're the same neuron and stimuli 
            ##      (ie mouse, session, stimuli)
            
            neuronArr_anmlSess = np.empty((getNumNeurons(df_trunc),minTPs-1)) # -1 for 0 indexing
            for stimLst in [trials_stimA, trials_stimB]:
                if np.array_equal(stimLst,trials_stimA) == True:
                    print('\nstimulus:', stimA)
                elif np.array_equal(stimLst,trials_stimB) == True:
                    print('\nstimulus:', stimB)
                else:
                    raise RuntimeError('unexpected trial concatenation condition occurred')
                print('selected minTPs: ', minTPs)
                
                ## create temporary sub matrix of concatenated cells for ONE stimulus
                tmp_neuronsArr_sameStim = np.empty((getNumNeurons(df_trunc),minTPs)) # for 0 indexing
                for trial in stimLst:
                    print("appending same neurons in trial: ", trial)
                    
                    ##  create temporary sub matrix of same trial all cells
                    tmp_neuronsArr_sameStim_sameTrial = np.empty((getNumNeurons(df_trunc),minTPs))
                    for neuron in getNeurons(df_trunc):
                        tmp_neuronInds = (df_trunc['trial']==trial) & (df_trunc['neuronID']==neuron)
                        tmp_neuronSeries = df_trunc.loc[tmp_neuronInds,'CaSignal']
                        tmp_neuronVec = tmp_neuronSeries.as_matrix()
                        tmp_neuronsArr_sameStim_sameTrial[neuron-1,:] = tmp_neuronVec
                    
                    ## append trials to right of same stim if not the first entry
                    if trial == stimLst[0]:
                        tmp_neuronsArr_sameStim = np.copy(tmp_neuronsArr_sameStim_sameTrial)
                    else:    
                        tmp_neuronsArr_sameStim = np.concatenate((tmp_neuronsArr_sameStim, tmp_neuronsArr_sameStim_sameTrial), axis=1)
                    print('same stim:', np.shape(tmp_neuronsArr_sameStim))
                
                ## append to bottom if not the first entry
                if np.array_equal(stimLst,trials_stimA):
                    neuronArr_anmlSess = tmp_neuronsArr_sameStim
                else:
                    neuronArr_anmlSess = np.concatenate((neuronArr_anmlSess,tmp_neuronsArr_sameStim), axis=0)
                print('bothStim', np.shape(neuronArr_anmlSess))
                
                

            print('\n')
        print('########\n')

print('total number of comparisons: ', totalNumComparisons)

('USS', 'baseline')
stimuli comparison num:  1
animal:  1
session: 1
USS baseline
trial IDs for each stimulus type [ 5 10 16] [ 1  8 15]
[[ 1117.  1116.  1117.]
 [  370.   363.  1476.]]
trial numbers match
checking approx num of time points
DISCARDED: variance in trial length is above the user's threshold
########

stimuli comparison num:  1
animal:  3
session: 1
USS baseline
trial IDs for each stimulus type [ 4 11 17] [ 1  8 15]
[[ 1267.  1127.  1123.]
 [  674.   682.   678.]]
trial numbers match
checking approx num of time points

stimulus: USS
selected minTPs:  674
appending same neurons in trial:  4
same stim: (30, 674)
appending same neurons in trial:  11
same stim: (30, 1348)
appending same neurons in trial:  17
same stim: (30, 2022)
bothStim (30, 2022)

stimulus: baseline
selected minTPs:  674
appending same neurons in trial:  1
same stim: (30, 674)
appending same neurons in trial:  8
same stim: (30, 1348)
appending same neurons in trial:  15
same stim: (30, 2022)
bothStim (60, 

same stim: (24, 1116)
appending same neurons in trial:  13
same stim: (24, 2232)
appending same neurons in trial:  17
same stim: (24, 3348)
bothStim (48, 3348)


session: 2
USS male
trial IDs for each stimulus type [ 4 11 21] [ 2 13 18]
[[ 1015.  1329.  1214.]
 [ 1166.  1113.  1167.]]
trial numbers match
checking approx num of time points

stimulus: USS
selected minTPs:  1015
appending same neurons in trial:  4
same stim: (25, 1015)
appending same neurons in trial:  11
same stim: (25, 2030)
appending same neurons in trial:  21
same stim: (25, 3045)
bothStim (25, 3045)

stimulus: male
selected minTPs:  1015
appending same neurons in trial:  2
same stim: (25, 1015)
appending same neurons in trial:  13
same stim: (25, 2030)
appending same neurons in trial:  18
same stim: (25, 3045)
bothStim (50, 3045)


session: 3
USS male
trial IDs for each stimulus type [ 6 13 16] [ 4 11 19]
[[ 1139.  1140.  1232.]
 [ 1134.  1114.  1168.]]
trial numbers match
checking approx num of time points

stimulus

USS mineral oil odor
trial IDs for each stimulus type [ 4 11] [ 3 12]
[[ 1137.  1113.]
 [ 1131.  1110.]]
trial numbers match
checking approx num of time points

stimulus: USS
selected minTPs:  1110
appending same neurons in trial:  4
same stim: (56, 1110)
appending same neurons in trial:  11
same stim: (56, 2220)
bothStim (56, 2220)

stimulus: mineral oil odor
selected minTPs:  1110
appending same neurons in trial:  3
same stim: (56, 1110)
appending same neurons in trial:  12
same stim: (56, 2220)
bothStim (112, 2220)


session: 2
USS mineral oil odor
trial IDs for each stimulus type [5 9] [ 6 10]
[[ 1133.  1124.]
 [ 1125.  1116.]]
trial numbers match
checking approx num of time points

stimulus: USS
selected minTPs:  1116
appending same neurons in trial:  5
same stim: (49, 1116)
appending same neurons in trial:  9
same stim: (49, 2232)
bothStim (49, 2232)

stimulus: mineral oil odor
selected minTPs:  1116
appending same neurons in trial:  6
same stim: (49, 1116)
appending same neurons

same stim: (57, 2024)
bothStim (114, 2024)


session: 2
USS peanut odor
trial IDs for each stimulus type [ 8 12] [4 9]
[[ 1022.  1018.]
 [ 1014.  1160.]]
trial numbers match
checking approx num of time points

stimulus: USS
selected minTPs:  1014
appending same neurons in trial:  8
same stim: (40, 1014)
appending same neurons in trial:  12
same stim: (40, 2028)
bothStim (40, 2028)

stimulus: peanut odor
selected minTPs:  1014
appending same neurons in trial:  4
same stim: (40, 1014)
appending same neurons in trial:  9
same stim: (40, 2028)
bothStim (80, 2028)


session: 3
USS peanut odor
trial IDs for each stimulus type [ 1 12] [ 3 11]
[[ 1015.  1053.]
 [ 1135.  1127.]]
trial numbers match
checking approx num of time points

stimulus: USS
selected minTPs:  1015
appending same neurons in trial:  1
same stim: (54, 1015)
appending same neurons in trial:  12
same stim: (54, 2030)
bothStim (54, 2030)

stimulus: peanut odor
selected minTPs:  1015
appending same neurons in trial:  3
same stim

USS rat
trial IDs for each stimulus type [ 6 12 16] [ 4  9 18]
[[  918.   894.  1068.]
 [ 1005.  1111.  1102.]]
trial numbers match
checking approx num of time points

stimulus: USS
selected minTPs:  894
appending same neurons in trial:  6
same stim: (35, 894)
appending same neurons in trial:  12
same stim: (35, 1788)
appending same neurons in trial:  16
same stim: (35, 2682)
bothStim (35, 2682)

stimulus: rat
selected minTPs:  894
appending same neurons in trial:  4
same stim: (35, 894)
appending same neurons in trial:  9
same stim: (35, 1788)
appending same neurons in trial:  18
same stim: (35, 2682)
bothStim (70, 2682)


session: 3
USS rat
trial IDs for each stimulus type [ 3 14 17] [ 5 10 21]
[[ 1019.  1087.  1156.]
 [ 1098.  1144.  1026.]]
trial numbers match
checking approx num of time points

stimulus: USS
selected minTPs:  1019
appending same neurons in trial:  3
same stim: (34, 1019)
appending same neurons in trial:  14
same stim: (34, 2038)
appending same neurons in trial:  1

...

same stim: (35, 681)
appending same neurons in trial:  9
same stim: (35, 1362)
appending same neurons in trial:  18
same stim: (35, 2043)
bothStim (70, 2043)


session: 3
baseline rat
trial IDs for each stimulus type [ 1  8 15] [ 5 10 21]
[[  677.   679.   694.]
 [ 1098.  1144.  1026.]]
trial numbers match
checking approx num of time points

stimulus: baseline
selected minTPs:  677
appending same neurons in trial:  1
same stim: (34, 677)
appending same neurons in trial:  8
same stim: (34, 1354)
appending same neurons in trial:  15
same stim: (34, 2031)
bothStim (34, 2031)

stimulus: rat
selected minTPs:  677
appending same neurons in trial:  5
same stim: (34, 677)
appending same neurons in trial:  10
same stim: (34, 1354)
appending same neurons in trial:  21
same stim: (34, 2031)
bothStim (68, 2031)


########

stimuli comparison num:  15
animal:  4
session: 1
baseline rat
trial IDs for each stimulus type [] [ 2 16]
[[   nan    nan]
 [ 1233.  1133.]]
DISCARDED: mismatching numbers of t

same stim: (21, 1114)
appending same neurons in trial:  11
same stim: (21, 2228)
appending same neurons in trial:  19
same stim: (21, 3342)
bothStim (42, 3342)


########

stimuli comparison num:  18
animal:  3
session: 1
female male
trial IDs for each stimulus type [ 7  9 19] [ 5 12 16]
[[ 1114.  1116.  1115.]
 [ 1123.  1108.  1138.]]
trial numbers match
checking approx num of time points

stimulus: female
selected minTPs:  1108
appending same neurons in trial:  7
same stim: (30, 1108)
appending same neurons in trial:  9
same stim: (30, 2216)
appending same neurons in trial:  19
same stim: (30, 3324)
bothStim (30, 3324)

stimulus: male
selected minTPs:  1108
appending same neurons in trial:  5
same stim: (30, 1108)
appending same neurons in trial:  12
same stim: (30, 2216)
appending same neurons in trial:  16
same stim: (30, 3324)
bothStim (60, 3324)


session: 2
female male
trial IDs for each stimulus type [ 3 13 17] [ 5 11 19]
[[  893.   941.  1015.]
 [  910.  1113.  1044.]]
trial n

same stim: (21, 1117)
appending same neurons in trial:  14
same stim: (21, 2234)
appending same neurons in trial:  17
same stim: (21, 3351)
bothStim (21, 3351)

stimulus: rat
selected minTPs:  1117
appending same neurons in trial:  2
same stim: (21, 1117)
appending same neurons in trial:  12
same stim: (21, 2234)
appending same neurons in trial:  21
same stim: (21, 3351)
bothStim (42, 3351)


########

stimuli comparison num:  22
animal:  3
session: 1
female rat
trial IDs for each stimulus type [ 7  9 19] [ 2 14 18]
[[ 1114.  1116.  1115.]
 [ 1147.  1211.  1112.]]
trial numbers match
checking approx num of time points

stimulus: female
selected minTPs:  1112
appending same neurons in trial:  7
same stim: (30, 1112)
appending same neurons in trial:  9
same stim: (30, 2224)
appending same neurons in trial:  19
same stim: (30, 3336)
bothStim (30, 3336)

stimulus: rat
selected minTPs:  1112
appending same neurons in trial:  2
same stim: (30, 1112)
appending same neurons in trial:  14
same 

same stim: (24, 1112)
appending same neurons in trial:  9
same stim: (24, 2224)
appending same neurons in trial:  21
same stim: (24, 3336)
bothStim (48, 3336)


session: 2
female toy
trial IDs for each stimulus type [ 8  9 20] [ 3 12 19]
[[ 1116.  1120.  1114.]
 [ 1131.  1114.  1113.]]
trial numbers match
checking approx num of time points

stimulus: female
selected minTPs:  1113
appending same neurons in trial:  8
same stim: (25, 1113)
appending same neurons in trial:  9
same stim: (25, 2226)
appending same neurons in trial:  20
same stim: (25, 3339)
bothStim (25, 3339)

stimulus: toy
selected minTPs:  1113
appending same neurons in trial:  3
same stim: (25, 1113)
appending same neurons in trial:  12
same stim: (25, 2226)
appending same neurons in trial:  19
same stim: (25, 3339)
bothStim (50, 3339)


session: 3
female toy
trial IDs for each stimulus type [ 3 14 17] [ 7 10 20]
[[ 1117.  1151.  1141.]
 [ 1116.  1163.  1141.]]
trial numbers match
checking approx num of time points

stim

same stim: (57, 2024)
bothStim (57, 2024)

stimulus: mineral oil odor
selected minTPs:  1012
appending same neurons in trial:  3
same stim: (57, 1012)
appending same neurons in trial:  16
same stim: (57, 2024)
bothStim (114, 2024)


session: 2
male mineral oil odor
trial IDs for each stimulus type [ 3 11] [ 5 14]
[[ 1016.  1009.]
 [ 1029.  1339.]]
trial numbers match
checking approx num of time points

stimulus: male
selected minTPs:  1009
appending same neurons in trial:  3
same stim: (40, 1009)
appending same neurons in trial:  11
same stim: (40, 2018)
bothStim (40, 2018)

stimulus: mineral oil odor
selected minTPs:  1009
appending same neurons in trial:  5
same stim: (40, 1009)
appending same neurons in trial:  14
same stim: (40, 2018)
bothStim (80, 2018)


session: 3
male mineral oil odor
trial IDs for each stimulus type [7 9] [ 2 14]
[[ 1022.  1022.]
 [ 1010.  1015.]]
trial numbers match
checking approx num of time points

stimulus: male
selected minTPs:  1010
appending same neuro

In [15]:
test = np.asarray([[0,1,2,3],[0,1,2,3]])
print(test)
print(np.shape(test))
test = np.asarray([0,1,2,3])
print(test)
print(np.shape(test))

[[0 1 2 3]
 [0 1 2 3]]
(2, 4)
[0 1 2 3]
(4,)


In [ ]:
#                     tmp_trialInds = (data['trial']==trial) 
#                     print(np.sum(tmp_trialInds))
#                     tmp_mat = data[tmp_trialInds].as_matrix()
#                     print(tmp_mat)
#                     print(type(tmp_neuronConcatArr))
#                     np.append(tmp_neuronConcatArr,tmp_mat)
#                     print(tmp_neuronConcatArr)

In [ ]:
testInds = (df['animal']==1) & (df['session']==1) & (df['stimulus']=='USS') & (df['trial']==5)
print(np.sum(testInds))

In [ ]:
np.unique(df_anmlSessStimA[df_anmlSessStimA['trial'] == trialA].tolist())

In [ ]:

# gb = df.groupby(['stimulusType']).get_group('rat')

stimGrouped = df.groupby('stimulusType')
ratGrpd = stimGrouped.get_group('rat')

print(ratGrpd)

print('reached end of file')

gb = df.groupby('stimulusType')

df_means = gb.apply(np.mean)
df_means

for stimType in gb

In [ ]:
ratData = df.loc[df['stimulusType'] == 'rat',:]
ussData = df.loc[df['stimulusType'] == 'USS',:]

print(ussData)

# slicing
df_big_force = df.loc[df['impact force (mN)'] > 1000, :]

df = pd.concat((df_low, df_high), axis=1)

# Specify indices we want (note parentheses holding each Boolean)
inds = (df['food density'] == 'high') & (df['cross-sectional area (sq micron)'] > 2000)

# Pull out areas
df.loc[inds, 'cross-sectional area (sq micron)']

df.corr()


# Rename the impact force column
df = df.rename(columns={'impact force (mN)': 'impf'})



# Write out DataFrame
df.to_csv('xa_combined.csv', index=False)